In [1]:
import openai
import os
import pandas as pd
import time
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from collections import defaultdict
import json
from langchain_text_splitters import RecursiveJsonSplitter

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_TRACING_V2 = os.getenv('LANGCHAIN_TRACING_V2')
LANGCHAIN_ENDPOINT = os.getenv('LANGCHAIN_ENDPOINT')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
LANGCHAIN_PROJECT = os.getenv('LANGCHAIN_PROJECT')

In [2]:
embedding_function = OpenAIEmbeddings()

# Create the language model
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.0)

analysis_prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    """
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{context}"
     """,
     ),
    ("human", "{question}")
    ]
)

In [10]:
def csv_files_to_json(file_paths, encoding='cp949'):
    # Initialize a default dictionary to store dictionaries for each ID
    json_data = defaultdict(lambda: defaultdict(list))
    
    # Iterate through each CSV file
    for file_path in file_paths:
        # Extract the file name without extension to use as a key
        file_key = file_path.split('/')[-1].split('.')[0]
        
        # Read the CSV file into a DataFrame with cp949 encoding
        df = pd.read_csv(file_path, encoding=encoding)
        
        # Group the data by 'id'
        for id, group in df.groupby('id'):
            # Convert the group data to a dictionary without the 'id' column
            record = group.drop(columns=['id']).to_dict(orient='records')
            # Append the record under the corresponding file name for the given id
            json_data[str(id)][file_key].extend(record)
    
    # Convert the default dictionary to a regular dictionary and then to a JSON string
    json_str = json.dumps(dict(json_data), ensure_ascii=False, indent=4)
    
    print(json_str)
        
    return json_str

In [11]:
def inbody_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/inbody.csv']
    
    inbody_json = csv_files_to_json(file_paths)
    json_data = json.loads(inbody_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 인바디 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
        
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    # texts = splitter.split_text(json_data=json_data)

    #벡터스토어 생성
    inbody_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    inbody_retriever = inbody_vectorstore.as_retriever()
    inbody_query = f"""
    너는 인바디 정보를 받아 각 환자마다 이를 토대로 추천운동, 추천식단, 건강정보에 중요한 정보들을 정리해야 한다.
    date값은 환자가 측정한 자신의 인바디 측정일이다.
    환자의 인바디 정보가 시간에 따른 변화에서 유의미한 정보가 있다면 정리하는 내용에 포함하라.
    """

    inbody_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    inbody_query,
     ),
    ("human", "{question}")
    ]
)
    
    inbody_chain = (
        {"context": inbody_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    inbody_response = inbody_chain.invoke(question)
    return inbody_response

In [12]:
def vital_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/vital.csv']
    
    vital_json = csv_files_to_json(file_paths)
    json_data = json.loads(vital_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 바이탈 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    vital_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    vital_retriever = vital_vectorstore.as_retriever()
    vital_query = f"""
    당신은 환자의 주요 생체 신호 정보를 추천운동, 추천식단, 건강정보에 중요한 정보들을 정리해야 한다. 
    환자의 생체 신호가 시간에 따른 변화에서 유의미한 정보가 있다면 정리하는 내용에 포함하라.
    """

    vital_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    vital_query,
     ),
    ("human", "{question}")
    ]
)
    
    vital_chain = (
        {"context": vital_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    vital_response = vital_chain.invoke(question)
    return vital_response

In [13]:
def patients_analysis(embedding_function, llm, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/patients.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/patients.csv']
    
    patients_json = csv_files_to_json(file_paths)
    json_data = json.loads(patients_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 환자 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    patients_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    patients_retriever = patients_vectorstore.as_retriever()
    
    patients_query = f"""
    context에 있는 정보를 정리하여 간단하게 출력하라. 현재 날짜는 :{time.strftime('%Y.%m.%d')}이다.
    환자의 나이는 현재 날짜 년도 - 출생년도이다.
    """
    
    patients_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    patients_query,
     ),
    ("human", "{question}")
    ]
)
    
    patients_chain = (
        {"context": patients_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    patients_response = patients_chain.invoke(question)
    return patients_response

In [14]:
def reports_analysis(embedding_function, llm, prompt, id, question):
    
    # file_paths = ['./data/inbody.csv', './data/reports.csv','./data/reports.csv','./data/vital.csv']
    file_paths = ['./data/col_value_change/reports.csv']
    
    reports_json = csv_files_to_json(file_paths)
    json_data = json.loads(reports_json)
    
    # 특정 id의 데이터 불러오기 (예: id가 '1'인 경우)
    target_id = id
    
    # 해당 id가 존재하는지 확인
    if target_id in json_data:
        id_data = json_data[target_id]
        print(f"{target_id}에 대한 설문 정보 분석 중...")
        json_output = json.dumps(id_data)
    else:
        print(f"ID {target_id} not found.")
    
    json_data = json.loads(json_output)
    
    splitter = RecursiveJsonSplitter(max_chunk_size=300)
    
    
    # The splitter can also output documents
    docs = splitter.create_documents(texts=[json_data])
    
    # or a list of strings
    texts = splitter.split_text(json_data=json_data)
    #벡터스토어 생성
    reports_vectorstore = FAISS.from_documents(docs, embedding_function)
    #retriever
    reports_retriever = reports_vectorstore.as_retriever()
    
    reports_query = f"""
    당신은 환자가 응답한 설문지에 대한 정보를 추천운동, 추천식단, 건강정보에 중요한 정보들을 정리해야 한다. 
    환자부모에 대한 응답 결과값도 유전력이 있는 질병이라면 이에 대한 점도 유의해야 돼.
    """

    reports_query += "{context}"
    prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    reports_query,
     ),
    ("human", "{question}")
    ]
)
    
    reports_chain = (
        {"context": reports_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    reports_response = reports_chain.invoke(question)
    return reports_response

In [15]:
def final_response(id, question):
    inbody_response = inbody_analysis(embedding_function, llm, analysis_prompt, id, question)
    vital_response = vital_analysis(embedding_function, llm, analysis_prompt, id, question)
    patients_response = patients_analysis(embedding_function, llm, id, question)
    reports_resonse = reports_analysis(embedding_function, llm, analysis_prompt, id, question)
    total_analysis = f"""
    환자 정보 :
    {patients_response}
    
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    
    설문조사 분석 :
    {reports_resonse} 
    """
    
    final_query = f"""
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{total_analysis}"""
    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
         final_query         
         ),
        ("human", "{question}")
    ])
    final_chain = (
        {"question": RunnablePassthrough()}
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    response = final_chain.invoke(question)
    print('-------------------------------------------------')
    return response

In [16]:
print(final_response('7','건강정보에 대한 종합적인 소견을 말해'))

{
    "0": {
        "inbody": [
            {
                "측정일자": "2024-06-30",
                "측정시간": "23:02:18",
                "체지방량(Kg)": 18.2,
                "체수분(L)": 34.7,
                "제지방량(Kg)": 47.3,
                "골격근량(Kg)": 25.9,
                "근육량(Kg)": 44.6,
                "체질량지수(kg/m^2)": 22.7,
                "체지방률(%)": 27.7,
                "복부지방률(허리둘레/엉덩이둘레)": 0.86,
                "골무기질량(Kg)": 2.73,
                "체세포량(Kg)": 30.5,
                "기초대사량(kcal)": 1392,
                "내장지방레벨": 7
            },
            {
                "측정일자": "2024-07-17",
                "측정시간": "7:59:14",
                "체지방량(Kg)": 17.5,
                "체수분(L)": 34.5,
                "제지방량(Kg)": 47.0,
                "골격근량(Kg)": 25.7,
                "근육량(Kg)": 44.3,
                "체질량지수(kg/m^2)": 22.4,
                "체지방률(%)": 27.1,
                "복부지방률(허리둘레/엉덩이둘레)": 0.86,
                "골무기질량(Kg)": 2.68,
                "체세포량(Kg)": 30.3,
          

KeyboardInterrupt: 

In [10]:
print(final_response('7','추천하는 운동을 알려줘'))

7에 대한 인바디 정보 분석 중...
7에 대한 바이탈 정보 분석 중...
7에 대한 환자 정보 분석 중...
7에 대한 설문 정보 분석 중...
종합 분석  
환자의 운동 추천 목록은 다양한 운동 형태를 포함하고 있으며, 각 운동은 특정한 건강 이점이 있습니다. 걷기는 심혈관 건강에 좋고, 조깅은 체력 증진과 스트레스 해소에 효과적입니다. 수영은 관절에 부담이 적으면서 전신 운동을 통해 근력과 유연성을 동시에 기를 수 있습니다. 요가는 유연성을 높이고 정신적인 안정에 기여하며, 근력 운동은 근육량을 증가시켜 기초 대사량을 높이는 데 도움을 줍니다. 이러한 운동들은 환자의 체력과 취향에 맞춰 선택할 수 있으며, 꾸준한 실천이 중요합니다.

종합적인 조언  
환자는 자신의 체력 수준과 개인적인 취향을 고려하여 위의 운동 중에서 선택하는 것이 좋습니다. 운동을 시작하기 전에 가벼운 스트레칭으로 몸을 풀고, 운동 후에는 충분한 휴식을 취하는 것이 중요합니다. 또한, 운동의 빈도와 강도를 점진적으로 증가시키며, 필요시 전문가의 조언을 받는 것이 바람직합니다. 건강한 식습관과 함께 운동을 병행하면 더욱 효과적인 결과를 얻을 수 있습니다.


In [11]:
print(final_response('7','추천하는 식단들을 설명해'))

7에 대한 인바디 정보 분석 중...
7에 대한 바이탈 정보 분석 중...
7에 대한 환자 정보 분석 중...
7에 대한 설문 정보 분석 중...
I do not know.
